# Notebook for transferring nodes

This notebook is intended to transfer the nodes from the finite-element positioning simulation o obtain the positioned model for further uses. 

Requirements: Python Version >= 3.8

How to use this notebook:
1. Open your simulation result (d3plots) in LS-PrePost
2. Export the new nodal coordinates of the last plot (Post -> Output)
    - Activate entire model
    - Activate nodal coordiantes
    - Activate Append St No:
    - Press Curr
    - Press Write
3. Name the exported file node.k
4. Copy the file in the same folder as this notebook
5. Run this notebook (Cell -> Run All)
6. Replace the nodes of the non-positioned model (e.g. for 50F-standing)
    - vivaplus_50F-standing_nodes.k nodes replaced with nodes of positioned_nodes.k file
    - vivaplus-50F-standing_reference_points.k nodes replaced with nodes of ref_points.k file
7. Now your positioned model should be ready to go



Copyright (C) 2020                                                                            
DI Christoph Leo                                                             
DI Dr. Corina Klug                                                           
                                                                           
CONTACT:                                                                             
Graz University of Technology                                                
Vehicle Safety Institute                                                     
Inffeldgasse 23 / I                                                          
8010 Graz                                                                    
email: christoph.leo@tugraz.at                                               
       corina.klug@tugraz.at     

In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import linecache
from glob import glob

In [13]:
input_dir = os.getcwd()
path_to_data = os.path.abspath(os.path.join(input_dir, "..\..\common"))

In [14]:
def search_string_in_files(file_names, list_of_string):
    line_numbers = []
    id_list=[]
    for file in file_names:

        line_number = 0    
        # Open the file in read only mode
        with open(file, 'r') as read_obj:
            # Read all lines in the file one by one
            for line in read_obj:
                line_number += 1
                # For each line, check if line contains any string from the list of strings
                for string_to_search in list_of_strings:
                    if string_to_search in line:
                    # If any string is found in line, then append that line along with line number in list
                        line_numbers.append((file,line_number+2,string_to_search))
                
    # Return list of tuples containing matched string, line numbers and lines where string is found
    return line_numbers

In [15]:
EXT = "*.k"
all_k_files = []
for path, subdir, files in os.walk(path_to_data):
    for file in glob(os.path.join(path, EXT)):
        all_k_files.append(file)     

In [16]:
id_list=[]
list_of_strings=['*CONSTRAINED_JOINT_SPHERICAL_ID','*CONSTRAINED_JOINT_REVOLUTE_ID']
line_numbers=search_string_in_files(all_k_files,list_of_strings)
line_numbers
for line_number in line_numbers:
    if '*CONSTRAINED_JOINT_SPHERICAL_ID' in line_number[2]:
        line=linecache.getline(line_number[0],line_number[1]).rstrip()
        id_list.append((line[3:10],line[13:20]))
    else:
        line=linecache.getline(line_number[0],line_number[1]).rstrip()
        id_list.append((line[3:10],line[13:20]))
        id_list.append((line[23:30],line[33:40]))

In [17]:
df=pd.read_table(os.path.join(input_dir, "node.k"),sep='\s+',skiprows=5,names=['id','x','y','z'])
df=df.round(10)
df=df[:-1].astype(str)



C:\Users\christoph_leo\AppData\Local\Continuum\anaconda3\envs\py38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
for ids in id_list:
    arr=df.loc[df['id'] == str(ids[0])].values
    df.loc[df['id'] == str(ids[1]),('x','y','z')]=arr[0][1:]
   

df['id']=df['id'].astype(int)

In [21]:
df['tc']=0
df['rd']=0
#df[(df['col1'] >= 1) & (df['col1'] <=1 )]
df_standing_nodes=df[(df['id'] <= 7268359)]
df_standing_nodes=pd.concat([df_standing_nodes,df[(df['id'] >= 9000010) & (df['id'] <= 9100019)]])
with open('positioned_nodes.k', 'w') as f:
    f.write('*KEYWORD\n*NODE\n')
    df_standing_nodes.to_string(f, col_space=(8,15,15,15,7,7),index=False,header=False)
    f.write('\n*END')
    f.close()
    
df_ref_points=df[(df['id'] >= 8110010) & (df['id'] <= 8983459)]
with open('ref_points.k', 'w') as f:
    f.write('*NODE\n$#   nid               x               y               z      tc      rc\n')
    df_ref_points.to_string(f, col_space=(8,15,15,15,7,7),index=False,header=False)
    f.write('\n*END')
    f.close()